<a href="https://colab.research.google.com/github/rrodrigo11/BostonHousingML/blob/main/House%20Prices%20-%20Regression%20Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook that contains basic handling values for data.

Using technique like :dropping columns with missing values, simple impute and complex impute

In [1]:
import pandas as pd

# Load data
train_data = pd.read_csv('../content/train.csv')
test_data = pd.read_csv('../content/test.csv')

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

train_data_target = train_data.SalePrice
train_data_predictors = train_data.drop(['SalePrice'], axis=1)

# I will avoid using non-numeric variables, because it will make my work easier.
train_data_numeric_predictors = train_data_predictors.select_dtypes(exclude=['object'])
test_data_numeric_predictors = test_data.select_dtypes(exclude=['object'])


In [2]:
X_train, X_test, y_train, y_test = train_test_split(train_data_numeric_predictors,
                                                    train_data_target,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=1)

def score_dataset(X_train, X_test, y_train, y_test):
    #print(y_train.describe().apply(lambda x: '{:.2f}'.format(x)))
    model = RandomForestRegressor(random_state = 1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds), model

*Droping missing values*


In [3]:
cols_with_missing = [col for col in X_train.columns
                                 if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_test  = X_test.drop(cols_with_missing, axis=1)
print("Mean Absolute Error from dropping columns with Missing Values:")
mae, drop_na_model = score_dataset(reduced_X_train, reduced_X_test, y_train, y_test)
print(mae)

Mean Absolute Error from dropping columns with Missing Values:
18235.67107305936


*Simple impute*

In [4]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = my_imputer.fit_transform(X_train)
imputed_X_test = my_imputer.transform(X_test)
print("Mean Absolute Error from Imputation:")
mae, simple_impute_model = score_dataset(imputed_X_train, imputed_X_test, y_train, y_test)
print(mae)

Mean Absolute Error from Imputation:
17772.564315068492


Complex impute

In [5]:
imputed_X_train_plus = X_train.copy()
imputed_X_test_plus = X_test.copy()

cols_with_missing = (col for col in X_train.columns
                                 if X_train[col].isnull().any())
for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull()
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus)
imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)

print("Mean Absolute Error from Imputation while Track What Was Imputed:")
mae, complex_impute_model = score_dataset(imputed_X_train_plus, imputed_X_test_plus, y_train, y_test)
print(mae)

Mean Absolute Error from Imputation while Track What Was Imputed:
17845.077534246575


After reviewing the mean absolute value of each test. The best result comes out of simple impute.
This will be the model used for this regression problem.

In [6]:
imputed_X_train = my_imputer.fit_transform(X_train)
imputed_test_data_predictors = my_imputer.transform(test_data_numeric_predictors)
test_data_preds = simple_impute_model.predict(imputed_test_data_predictors)


In [8]:
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_data_preds})
print(output)

        Id  SalePrice
0     1461  128016.58
1     1462  153953.50
2     1463  185536.07
3     1464  182411.82
4     1465  199771.85
...    ...        ...
1454  2915   83892.00
1455  2916   84533.00
1456  2917  158861.27
1457  2918  118162.75
1458  2919  244000.85

[1459 rows x 2 columns]
